# Servidor TCP en C 

Compilación y ejecución de un servidor TCP en C en Linux.
Tener instalado `gcc` antes de continuar.

In [ ]:
# Verificamos si gcc está instalado
!gcc --version

## 1. Creación del archivo fuente
Guardar el código en un archivo llamado `servidor.c`.

In [ ]:
%%writefile servidor.c
#include <stdio.h>
#include <netdb.h>
#include <netinet/in.h>
#include <stdlib.h>
#include <string.h>
#include <sys/socket.h>
#include <sys/types.h>
#include <unistd.h> // read(), write(), close()

#define MAX 80
#define PORT 8080
#define SA struct sockaddr

void func(int connfd) {
    char buff[MAX];
    int n;
    for (;;) {
        bzero(buff, MAX);
        read(connfd, buff, sizeof(buff));
        printf("Del cliente: %s\t: ", buff);
        bzero(buff, MAX);
        n = 0;
        while ((buff[n++] = getchar()) != '\n');
        write(connfd, buff, sizeof(buff));
        if (strncmp("SALIR", buff, 4) == 0) {
            printf("Salgo del servidor...\n");
            break;
        }
    }
}

int main() {
    int sockfd, connfd, len;
    struct sockaddr_in servaddr, cli;
    
    sockfd = socket(AF_INET, SOCK_STREAM, 0);
    if (sockfd == -1) {
        printf("Falla la creación del socket...\n");
        exit(0);
    }
    printf("Socket creado...\n");
    bzero(&servaddr, sizeof(servaddr));
    
    servaddr.sin_family = AF_INET;
    servaddr.sin_addr.s_addr = htonl(INADDR_ANY);
    servaddr.sin_port = htons(PORT);
    
    if ((bind(sockfd, (SA*)&servaddr, sizeof(servaddr))) != 0) {
        printf("Falla socket bind ...\n");
        exit(0);
    }
    printf("Se hace el socket bind ..\n");
    
    if ((listen(sockfd, 5)) != 0) {
        printf("Falla el listen ...\n");
        exit(0);
    }
    printf("Servidor en modo escucha ...\n");
    len = sizeof(cli);
    
    connfd = accept(sockfd, (SA*)&cli, &len);
    if (connfd < 0) {
        printf("Falla al aceptar datos en el servidor...\n");
        exit(0);
    }
    printf("El servidor acepta al cliente ...\n");
    
    func(connfd);
    close(sockfd);
}

## 2. Compilar el código
Compilar el código para verificar que no haya errores.

In [ ]:
!gcc servidor.c -o servidor

Si la compilación fue exitosa, podemos ejecutar el servidor.

In [ ]:
# Ejecutamos el servidor en segundo plano . ATENCION: ejecutar por fuera del Notebook desde una terminal
!./servidor &

## 3. Probar el servidor
Para conectarnos al servidor, usar `telnet` o `netcat` desde otra terminal.
Ejecutar el siguiente comando en una terminal aparte para conectarse al servidor:
```
nc 127.0.0.1 8080
```
Luego, escribir un mensaje y presionar Enter para enviarlo al servidor.

## P4. Detener el servidor
De ser necesario usar este comando:
```
!pkill servidor
```